<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

Suppose we wish to test the null hypothesis that the mean of a population
is equal to 0.5. We choose a confidence level of 99%; that is, we will
reject the null hypothesis in favor of the alternative if the p-value is
less than 0.01.

When testing random variates from the standard uniform distribution, which
has a mean of 0.5, we expect the data to be consistent with the null
hypothesis most of the time.


In [ ]:
import numpy as np
from scipy import stats
rng = np.random.default_rng()
rvs = stats.uniform.rvs(size=50, random_state=rng)
stats.ttest_1samp(rvs, popmean=0.5)

TtestResult(statistic=2.456308468440, pvalue=0.017628209047638, df=49)

As expected, the p-value of 0.017 is not below our threshold of 0.01, so
we cannot reject the null hypothesis.

When testing data from the standard *normal* distribution, which has a mean
of 0, we would expect the null hypothesis to be rejected.


In [ ]:
rvs = stats.norm.rvs(size=50, random_state=rng)
stats.ttest_1samp(rvs, popmean=0.5)

TtestResult(statistic=-7.433605518875, pvalue=1.416760157221e-09, df=49)

Indeed, the p-value is lower than our threshold of 0.01, so we reject the
null hypothesis in favor of the default "two-sided" alternative: the mean
of the population is *not* equal to 0.5.

However, suppose we were to test the null hypothesis against the
one-sided alternative that the mean of the population is *greater* than
0.5. Since the mean of the standard normal is less than 0.5, we would not
expect the null hypothesis to be rejected.


In [ ]:
stats.ttest_1samp(rvs, popmean=0.5, alternative='greater')

TtestResult(statistic=-7.433605518875, pvalue=0.99999999929, df=49)

Unsurprisingly, with a p-value greater than our threshold, we would not
reject the null hypothesis.

Note that when working with a confidence level of 99%, a true null
hypothesis will be rejected approximately 1% of the time.


In [ ]:
rvs = stats.uniform.rvs(size=(100, 50), random_state=rng)
res = stats.ttest_1samp(rvs, popmean=0.5, axis=1)
np.sum(res.pvalue < 0.01)

1

Indeed, even though all 100 samples above were drawn from the standard
uniform distribution, which *does* have a population mean of 0.5, we would
mistakenly reject the null hypothesis for one of them.

`ttest_1samp` can also compute a confidence interval around the population
mean.


In [ ]:
rvs = stats.norm.rvs(size=50, random_state=rng)
res = stats.ttest_1samp(rvs, popmean=0)
ci = res.confidence_interval(confidence_level=0.95)
ci

ConfidenceInterval(low=-0.3193887540880017, high=0.2898583388980972)

The bounds of the 95% confidence interval are the
minimum and maximum values of the parameter `popmean` for which the
p-value of the test would be 0.05.


In [ ]:
res = stats.ttest_1samp(rvs, popmean=ci.low)
np.testing.assert_allclose(res.pvalue, 0.05)
res = stats.ttest_1samp(rvs, popmean=ci.high)
np.testing.assert_allclose(res.pvalue, 0.05)

Under certain assumptions about the population from which a sample
is drawn, the confidence interval with confidence level 95% is expected
to contain the true population mean in 95% of sample replications.


In [ ]:
rvs = stats.norm.rvs(size=(50, 1000), loc=1, random_state=rng)
res = stats.ttest_1samp(rvs, popmean=0)
ci = res.confidence_interval()
contains_pop_mean = (ci.low < 1) & (ci.high > 1)
contains_pop_mean.sum()

953